In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os, gc, json, re

from sklearn.cluster import KMeans
from keplergl import KeplerGl

# KeplerGL
- https://kepler.gl/
- jupyter user guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md

# Some functions

In [ ]:
def clean_income_data(path, file_name):
    df_income = pd.read_csv(path + file_name)

    cols = ['GEO_ID', 'NAME']
    update_col_names = {}
    lower_level = [] # for calculating a weighted average
    for col_name, col_data in df_income.head(1).iteritems():
        real_name = col_data.values[0]
        if 'Estimate' in real_name:
            income_level = re.findall(r'\d+(?:,\d+)?', real_name)
            if len(income_level) == 1:
                if 'less than' in real_name.lower():
                    new_name = 'less than ' + income_level[0]
                    lower_num = 0
                elif 'more' in real_name.lower():
                    new_name = income_level[0] + ' or more'
                    lower_num = int(income_level[0].replace(',', ''))
            if len(income_level) == 2:
                new_name = income_level[0] + ' to ' + income_level[1]
                lower_num = int(income_level[0].replace(',', ''))

            if len(income_level) > 0:
                lower_level.append(lower_num)
                update_col_names.update({col_name: new_name})
                cols.append(col_name)

    df_income_clean = df_income[cols].rename(update_col_names, axis = 1).drop(0).reset_index(drop = True)
    income_level_cols = [c for c in list(df_income_clean) if c not in ['GEO_ID', 'NAME']]
    row_totals = df_income_clean[income_level_cols].astype(int).sum(axis = 1)
    weighted_sums = df_income_clean[income_level_cols].astype(int).values * np.array(lower_level)
    weighted_avgs = weighted_sums.sum(axis = 1) / row_totals
    df_income_clean['weighted_avg_income'] = weighted_avgs
    df_income_clean['GEO_ID'] = df_income_clean['GEO_ID'].str.replace('1500000US', '')
    # clean the weighted_avg_income column for display
    df_income_clean['Weighted Average Income'] = df_income_clean['weighted_avg_income'].apply('${:,.2f}'.format)
    
    return df_income_clean

def get_census_income():
    income_files = {
        '2018': 'ACSDT5Y2018.B19001_data_with_overlays_2020-02-13T163519.csv',
        '2017': 'ACSDT5Y2017.B19001_data_with_overlays_2020-02-13T163519.csv',
        '2016': 'ACSDT5Y2016.B19001_data_with_overlays_2020-02-13T163519.csv',
        '2015': 'ACSDT5Y2015.B19001_data_with_overlays_2020-02-13T163519.csv',
        '2014': 'ACSDT5Y2014.B19001_data_with_overlays_2020-02-13T163519.csv',
        '2013': 'ACSDT5Y2015.B19001_data_with_overlays_2020-02-13T163519.csv',
    }

    path = './data/Census Bureau/Income/productDownload_2020-02-13T163538/'
    df_final = pd.DataFrame()
    for year, file_name in income_files.items():
        df_current = clean_income_data(path, file_name)
        df_current['Year'] = pd.to_datetime(str(year), format = '%Y')
        df_final = df_final.append(df_current, sort = False).reset_index(drop = True)
        
    df_final = df_final.rename({'GEO_ID': 'GEOID'}, axis = 1)
    df_final = df_final.sort_values(by = ['Year'], ascending = False).reset_index(drop = True)
    return df_final

def get_spatial_df_from_clusters(df, min_cluster_observations = 10, crs = 'EPSG:4326'):
    xy_clusters = df[['x', 'y', 'clusters']].groupby('clusters').agg(['mean', 'std', 'count']).reset_index()
    xy_clusters.columns = [(e[0] + ' ' + e[1]).strip() for e in list(xy_clusters)]
    xy_clusters = xy_clusters.loc[xy_clusters['x count'] >= 24].reset_index(drop = True)
    geometry = gpd.points_from_xy(xy_clusters['x mean'], xy_clusters['y mean'])
    gdf_clusters = gpd.GeoDataFrame(xy_clusters, geometry = geometry)
    gdf_clusters.crs = crs
    
    gdf_clusters = gdf_clusters.rename({'y count': 'count'}, axis = 1)
    gdf_clusters = gdf_clusters.drop(['x std', 'y std', 'x count'], axis = 1)
    return gdf_clusters

# Starbucks
- data: https://www.kaggle.com/starbucks/store-locations/version/1

In [ ]:
path = './data/Starbucks/'
df = pd.read_csv(path + 'directory.csv')
df.head()

In [ ]:
geometries = gpd.points_from_xy(df['Longitude'], df['Latitude'])
gdf_starbucks = gpd.GeoDataFrame(df, geometry = geometries)
gdf_starbucks.crs = 'EPSG:4326'

In [ ]:
map_1 = KeplerGl(height = 500, data = {'starbucks': gdf_starbucks})
map_1

# Census

### Income Data
- data: https://data.census.gov/cedsci/

In [ ]:
# the following function cleans and formats the original csv downloads
df_income = get_census_income()
df_income.head(4)

In [ ]:
# this will plot the distribution of average income in the data
df_income['weighted_avg_income'].plot.hist()

### Census Tract Map
- shapefile: https://catalog.data.gov/dataset/tiger-line-shapefile-2017-state-california-current-block-group-state-based

In [ ]:
shape_path = './data/Census Tracts and Block Groups/tl_2017_06_bg/'
shape_file = 'tl_2017_06_bg.shp'
gdf = gpd.GeoDataFrame.from_file(shape_path + shape_file)
gdf.head(3)

In [ ]:
cols = ['GEOID', 'geometry']

# change the crs and filter to Fresno County
gdf = gdf.to_crs(epsg = '4326')
gdf_fresno = gdf[cols].loc[gdf['COUNTYFP'] == '019']

In [ ]:
map_1 = KeplerGl(height = 500, data = {'fresno_census_tracts': gdf_fresno})
map_1

In [ ]:
income_cols = ['GEOID', 'weighted_avg_income', 'Weighted Average Income', 'Year']
gdf_merged_income = gdf_fresno.merge(df_income[income_cols], on = 'GEOID')
gdf_merged_income.head()

In [ ]:
map_1 = KeplerGl(height = 500, data = {'fresno_income': gdf_merged_income})
map_1

In [ ]:
sbx_cols = ['Store Number', 'Store Name', 'City', 'geometry']
local_starbucks = gdf_starbucks[sbx_cols].loc[gdf_starbucks['City'].isin(['Fresno', 'Clovis'])]
local_starbucks.head()

### Spatial Join
- with geopandas.sjoin()
    - https://geopandas.org/reference/geopandas.sjoin.html
- be sure the coordinate systems are the same

In [ ]:
sbx_and_income = gpd.sjoin(local_starbucks, gdf_merged_income)
sbx_and_income.head()

In [ ]:
sbx_and_income.loc[sbx_and_income['Year'].dt.year == 2018].groupby(['City']).agg({'weighted_avg_income': 'mean'})

In [ ]:
cols = ['Store Name', 'City', 'weighted_avg_income', 'Weighted Average Income', 'Year', 'geometry']
map_1 = KeplerGl(height = 500, 
                 data = {'fresno_income': sbx_and_income.loc[sbx_and_income['Year'].dt.year == 2018, cols]})
map_1

# The Power of Geospatial Data

In [ ]:
file = './data/location_shapefile/Location_Tracking.shp'
gdf = gpd.GeoDataFrame.from_file(file)
gdf = gdf.to_crs(epsg = '4326')

In [ ]:
x = gdf.geometry.x
y = gdf.geometry.y
xy_data = pd.DataFrame({'x': x, 'y': y})
xy_data = xy_data.reset_index()
xy_data.plot(x = 'x', y= 'y', kind = 'scatter')

### Filtering down the data
- First we will find the minimum distance of each point to any other point
- Then we will filter all points to be within one standard deviation of the mean of those minimum distances
- This essentially tries to filter points to existing clusters or groups

In [ ]:
xy_data_cartesian = xy_data.copy()
xy_data_cartesian['key'] = 1
xy_data_cartesian = xy_data_cartesian.merge(xy_data_cartesian, on = 'key', suffixes = ('_1', '_2'))
xy_data_cartesian = xy_data_cartesian.loc[xy_data_cartesian['index_1'] != xy_data_cartesian['index_2']].reset_index(drop = True)
xy_data_cartesian['distance'] = np.sqrt(((xy_data_cartesian[['x_1', 'y_1']].values - xy_data_cartesian[['x_2', 'y_2']].values)**2).sum(axis = 1))

xy_data_min_d = xy_data_cartesian.groupby('index_1').agg({'distance': 'min'}).reset_index()
mean_min_d = xy_data_min_d['distance'].mean()
std_min_d = xy_data_min_d['distance'].std()

xy_data_filtered = xy_data.merge(xy_data_min_d, left_on = 'index', right_on = 'index_1').drop(['index', 'index_1'], axis = 1)
xy_data_filtered = xy_data_filtered.loc[np.abs(xy_data_filtered['distance'] - mean_min_d) <= 1*std_min_d]

xy_data_filtered.plot(x = 'x', y= 'y', kind = 'scatter')

# Clustering
- https://scikit-learn.org/stable/modules/clustering.html
- the clustering algorithm will attempt to give each group a distinct label

In [ ]:
model = KMeans(n_clusters = 4, random_state = 0)
clusters = model.fit_predict(xy_data_filtered[['x', 'y']])
xy_data_filtered['clusters'] = clusters
xy_data_filtered.plot.scatter(x = 'x', y = 'y', c = 'clusters', colormap = 'plasma')

In [ ]:
gdf_clusters = get_spatial_df_from_clusters(xy_data_filtered, min_cluster_observations = 24)
gdf_clusters

In [ ]:
map_1 = KeplerGl(height = 500, data = {'location_points': gdf_clusters})

In [ ]:
map_1

In [ ]:
shp_file = './data/Fresno County/Parcels/Fresno_Parcels.shp'
gdf_parcels = gpd.read_file(shp_file)
gdf_parcels_fresno = gdf_parcels.loc[gdf_parcels['AGENCY_COD'] == 'FR']
gdf_parcels_fresno = gdf_parcels_fresno.to_crs(epsg=4326)

# refresh map above and check the layers
# when adding data to a map, it might not be displayed on the map
# in that case it will have to be done from within the map
map_1.add_data(data = gdf_parcels_fresno, name = 'fresno_parcels')

# this will load a map config file if you have one
# after changing the map to your liking, you can save map.config into a json file to load later
# with open('fresno_parcel_and_location_map_config.json', 'r') as f:
#     map_config = json.load(f)
# map_1.config = map_config